In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline 

In [33]:
uniq_id_cust    = pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_customers_dataset.csv')
orders          = pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_orders_dataset.csv')
items           = pd.read_csv ('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-lesihina/first_project/olist_order_items_dataset.csv')

#### Задание 4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? 
Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели.

In [35]:
#объединим таблицу с id пользователей и таблицу с заказами
join_id_orders= uniq_id_cust.merge(orders, how = 'outer', on ='customer_id')

In [36]:
# привели к формуту даты
join_id_orders ['order_approved_at'] = pd.to_datetime(join_id_orders ['order_approved_at'])

In [37]:
# перевели в год, в котором сделан заказ и оплачен
join_id_orders ['year_approved'] = join_id_orders.order_approved_at.dt.year

In [38]:
# перевели в месяц, в котором сделан заказ и оплачен
join_id_orders ['month_approved'] = join_id_orders.order_approved_at.dt.month

In [39]:
join_id_orders\
    .groupby (['year_approved','month_approved'])\
    .agg ({'customer_unique_id': 'count'})

customer_unique_id
year_approved month_approved                    
2016.0        9.0                              1
              10.0                           320
              12.0                             1
2017.0        1.0                            760
              2.0                           1765
              3.0                           2689
              4.0                           2374
              5.0                           3693
              6.0                           3252
              7.0                           3974
              8.0                           4348
              9.0                           4301
              10.0                          4590
              11.0                          7395
              12.0                          5832
2018.0        1.0                           7187
              2.0                           6706
              3.0                           7288
              4.0                           6778
              5.0                           7066
              6.0                           6164
              7.0                           6176
              8.0                           6620
              9.0                              1

По таблице выше видно,что в 2016 году и в сентябре 2018 года очень мало заказов. Предположительно, это связано с тем, что выборка не полная, то есть в 2016 году данные только начинали собирать.
Та же ситуация с сентябрем 2018 г. - скорее всего сбор данных прекратился, поэтому там была всего одна покупка.
Как вывод - при наличии точной информации о старте сбора данных мы могли бы исключить данные значения
(2016 год и 9 месяц 2018 года). В рассчетах ниже все данные остаются.

In [41]:
# Отберем все покупки,кроме созданных, выставлен счет (неоплачен), недоступных и отмененных
right_orders = join_id_orders\
    .query('order_status != ["unavailable", "created", "invoiced", "canceled"]')\
    .groupby (['customer_unique_id','month_approved'])\
    .agg ({'order_id': 'count'})

In [42]:
right_orders

,,order_id
customer_unique_id,month_approved,
0000366f3b9a7992bf8c76cfdf3221e2,5.0,1
0000b849f77a49e4a4ce2b2a4ca5be3f,5.0,1
0000f46a3911fa3c0805444483337064,3.0,1
0000f6ccb0745a6a4b88665a16c9f078,10.0,1
0004aac84e0df4da2b147fca70cf8255,11.0,1
...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,6.0,1
fffea47cd6d3cc0a88bd621562a9d061,12.0,1
ffff371b4d645b6ecea244b27531430a,2.0,1


In [43]:
# сбросим индексы
right_orders = right_orders.reset_index()

In [44]:
'''
Месяц в котором 30 дней (апрель, июнь, сентябрь, ноябрь) - 30/7 = 4.29 (4 недели и 2 дня); 
Месяц в котором 31 день (январь, март, май, июль, август, октябрь, декабрь) - 31/7 = 4.43 (4 недели и 3 дня);
Месяц в котором 28 дней (февраль) - 28/7 = 4 недели (високосных годов нет, а в 2016 нет февраля)
'''

In [45]:
# создадим словарь для подсчета недель в месяцах
di = {
1: 4.43,
2: 4,
3: 4.43,
4: 4.29,
5: 4.43,
6: 4.29,
7: 4.43,
8: 4.43,
9: 4.29,
10: 4.43,
11: 4.29,
12: 4.43
}

In [46]:
# создадим столбец с информацией о том,сколько недель в каждом месяце
right_orders ['weeks'] = right_orders['month_approved'].map(di)

In [47]:
right_orders

,customer_unique_id,month_approved,order_id,weeks
0,0000366f3b9a7992bf8c76cfdf3221e2,5.0,1,4.43
1,0000b849f77a49e4a4ce2b2a4ca5be3f,5.0,1,4.43
2,0000f46a3911fa3c0805444483337064,3.0,1,4.43
3,0000f6ccb0745a6a4b88665a16c9f078,10.0,1,4.43
4,0004aac84e0df4da2b147fca70cf8255,11.0,1,4.29
...,...,...,...,...
96497,fffcf5a5ff07b0908bd4e2dbc735a684,6.0,1,4.29
96498,fffea47cd6d3cc0a88bd621562a9d061,12.0,1,4.43
96499,ffff371b4d645b6ecea244b27531430a,2.0,1,4.00
96500,ffff5962728ec6157033ef9805bacc48,5.0,1,4.43


In [48]:
# И создадим колонку с информацией о том, сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)
right_orders ['orders_by_week'] = right_orders['order_id'] / right_orders['weeks']

In [49]:
# сколько у каждого из пользователей в среднем покупок в неделю (в разрезе по месяцам)
right_orders.sort_values('customer_unique_id', ascending=True)

,customer_unique_id,month_approved,order_id,weeks,orders_by_week
0,0000366f3b9a7992bf8c76cfdf3221e2,5.0,1,4.43,0.225734
1,0000b849f77a49e4a4ce2b2a4ca5be3f,5.0,1,4.43,0.225734
2,0000f46a3911fa3c0805444483337064,3.0,1,4.43,0.225734
3,0000f6ccb0745a6a4b88665a16c9f078,10.0,1,4.43,0.225734
4,0004aac84e0df4da2b147fca70cf8255,11.0,1,4.29,0.233100
...,...,...,...,...,...
96497,fffcf5a5ff07b0908bd4e2dbc735a684,6.0,1,4.29,0.233100
96498,fffea47cd6d3cc0a88bd621562a9d061,12.0,1,4.43,0.225734
96499,ffff371b4d645b6ecea244b27531430a,2.0,1,4.00,0.250000
96500,ffff5962728ec6157033ef9805bacc48,5.0,1,4.43,0.225734


#### А теперь посмотрим то же самое, только без разбивки на месяца

In [51]:
# создадим столбец с информацией о том,сколько недель в каждом месяце
join_id_orders ['weeks'] = join_id_orders['month_approved'].map(di)

In [52]:
# отберем все покупки,кроме созданных, выставлен счет (неоплачен), недоступных и отмененных
right_orders_weeks = join_id_orders\
    .query('order_status != ["unavailable", "created", "invoiced", "canceled"]')\
    .groupby (['customer_unique_id'])\
    .agg ({'order_id': 'count', 'weeks': 'sum'})

In [58]:
right_orders_weeks.sort_values('order_id', ascending=False)

,order_id,weeks,count_by_week
customer_unique_id,,,
8d50f5eadf50201ccdcedfb9e2ac8455,16,70.46,0.227079
3e43e6105506432c953e165fb2acf44c,9,38.01,0.236780
1b6c7548a2a1f9037c1fd3ddfed95f33,7,30.30,0.231023
6469f99c1f9dfae7733b25662e7f1782,7,30.59,0.228833
ca77025e7201e3b30c44b472ff346268,7,29.87,0.234349
...,...,...,...
56546b71f8bf254f860014fae431af6d,1,4.00,0.250000
565427a96f0865bacef2a6b400197b46,1,4.29,0.233100
565283fe12fa1a894ef451c4bfb391ed,1,4.43,0.225734


In [54]:
right_orders_weeks ['count_by_week'] = right_orders_weeks['order_id']/right_orders_weeks['weeks']

In [57]:
right_orders_weeks.sort_values('order_id', ascending=False)

,order_id,weeks,count_by_week
customer_unique_id,,,
8d50f5eadf50201ccdcedfb9e2ac8455,16,70.46,0.227079
3e43e6105506432c953e165fb2acf44c,9,38.01,0.236780
1b6c7548a2a1f9037c1fd3ddfed95f33,7,30.30,0.231023
6469f99c1f9dfae7733b25662e7f1782,7,30.59,0.228833
ca77025e7201e3b30c44b472ff346268,7,29.87,0.234349
...,...,...,...
56546b71f8bf254f860014fae431af6d,1,4.00,0.250000
565427a96f0865bacef2a6b400197b46,1,4.29,0.233100
565283fe12fa1a894ef451c4bfb391ed,1,4.43,0.225734


In [32]:
join_id_orders.query ('customer_unique_id == "c4c0011e639bdbcf26059ddc38bd3c18"')

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,year_approved,month_approved,weeks
8970,0bf35cac6cc7327065da879e2d90fae8,c4c0011e639bdbcf26059ddc38bd3c18,13225,varzea paulista,SP,d77031d6a3c8a52f019764e68f211c69,delivered,2017-02-18 11:04:19,NaT,2017-02-23 07:23:36,2017-03-02 16:15:23,2017-03-22 00:00:00,NaN,NaN,NaN


#### Ответ: 

Значение покупок с разбивкой по месяцам - в таблице: right_orders, столбец orders_by_week
Среднее значение покупок по каждому пользователю - в таблице: right_orders_weeks,стоблец count_by_week

Вывод:
При отсутствии доп разбивки по месяцам можно увидеть более общую картину